In [1]:
import pandas as pd
import numpy as np

In [2]:
data = "WELFake_Dataset.csv"
df = pd.read_csv(data)
df.head()


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [3]:
df.shape


(72134, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [5]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [6]:
df['label'].value_counts()


label
1    37106
0    35028
Name: count, dtype: int64

In [7]:
df["title"] = df["title"].fillna("")
df["text"] = df["text"].fillna("")
df["content"] = df["title"] + " " + df["text"]

In [8]:
df = df[df["content"].str.strip().astype(bool)].reset_index(drop=True)

In [9]:
df

,Unnamed: 0,title,text,label,content
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,1,,Did they post their votes for Hillary already?,1,Did they post their votes for Hillary already?
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...
...,...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0,Russians steal research on Trump in hack of U....
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1,WATCH: Giuliani Demands That Democrats Apolog...
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0,Migrants Refuse To Leave Train At Refugee Camp...
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0,Trump tussle gives unpopular Mexican leader mu...


Preprocessing

In [10]:
import re
import string

def preprocess_text(text: str) -> str:
    text = str(text).lower()
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"<.*?>", " ", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = " ".join(text.split())

    return text



In [11]:
df["clean"] = df["content"].apply(preprocess_text)

In [12]:
df[["content", "clean"]].head()

,content,clean
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,law enforcement on high alert following threat...
1,Did they post their votes for Hillary already?,did they post their votes for hillary already
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,unbelievable obama’s attorney general says mos...
3,"Bobby Jindal, raised Hindu, uses story of Chri...",bobby jindal raised hindu uses story of christ...
4,SATAN 2: Russia unvelis an image of its terrif...,satan russia unvelis an image of its terrifyin...


Feature extraction

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=10000,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.95,
    sublinear_tf=True,
)

X = vectorizer.fit_transform(df["clean"])
y = df["label"]


In [14]:
df.shape

(72134, 6)

train-test split

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [16]:
X_train.shape[0]

57707

In [17]:
X_test.shape[0]

14427

Model training 


1. logistic regression

In [18]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1500, C=1.5, solver="saga", random_state=42, n_jobs=-1)

lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

In [19]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)



accuracy = accuracy_score(y_test, y_pred_lr)
precision = precision_score(y_test, y_pred_lr)
recall = recall_score(y_test, y_pred_lr)
f1 = f1_score(y_test, y_pred_lr)

y_proba_lr = lr.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_proba_lr)

In [20]:
print(accuracy)
print(precision)
print(recall)
print(f1)
print(y_proba_lr)
print(roc_auc)


0.9598669161987939
0.9554046858359957
0.9671203341867672
0.9612268130985067
[0.85198397 0.53400918 0.0144927  ... 0.99315422 0.78197417 0.01112536]
0.9921777445039794


In [21]:
from sklearn.model_selection import cross_val_score

cv = cross_val_score(lr, X_train, y_train, cv=5, scoring="f1", n_jobs=-1)
print(cv.mean(), "±", cv.std())

0.9612029007802295 ± 0.0016710767704381654


In [22]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix)

print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, target_names=["Real", "Fake"]))

[[6671  335]
 [ 244 7177]]
              precision    recall  f1-score   support

        Real       0.96      0.95      0.96      7006
        Fake       0.96      0.97      0.96      7421

    accuracy                           0.96     14427
   macro avg       0.96      0.96      0.96     14427
weighted avg       0.96      0.96      0.96     14427



2. training Linear svm model

In [23]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svm = CalibratedClassifierCV(
    LinearSVC(C=1.0, max_iter=2000, random_state=42),
    cv=3
)

svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)

In [24]:
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm))
print("Recall:", recall_score(y_test, y_pred_svm))
print("F1:", f1_score(y_test, y_pred_svm))

y_proba_svm = svm.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_proba_svm))

Accuracy: 0.9654813890621751
Precision: 0.9622112431566298
Recall: 0.9710281633203073
F1: 0.9665995975855131
ROC-AUC: 0.9936403867045565


3. Training decision tree model

In [25]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=50, min_samples_split=5, min_samples_leaf=3, random_state=42)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)

In [26]:
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Precision:", precision_score(y_test, y_pred_dt))
print("Recall:", recall_score(y_test, y_pred_dt))
print("F1:", f1_score(y_test, y_pred_dt))

y_proba_dt = dt.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_proba_dt))

Accuracy: 0.9376169681846538
Precision: 0.9290696144229504
Recall: 0.9513542649238647
F1: 0.9400798934753661
ROC-AUC: 0.9313183363765858


4. training Random Forest model

In [27]:
from sklearn.ensemble import (RandomForestClassifier)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=80,
    min_samples_split=4,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

In [28]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1:", f1_score(y_test, y_pred_rf))

y_proba_rf = rf.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_proba_rf))

Accuracy: 0.9568170790878214
Precision: 0.942232630757221
Recall: 0.9758792615550465
F1: 0.9587608393459985


ROC-AUC: 0.9931495182503396


Seeing the results : Linear SVM model wins in almost every metric (best accuracy (96.55%) , best precsion(96.22%) , best F1(96.66%) and best ROC-AUC (96.36%))

In [29]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred_final = svm.predict(X_test)

print(confusion_matrix(y_test, y_pred_final))
print(classification_report(y_test, y_pred_final, target_names=["Real", "Fake"]))

[[6723  283]
 [ 215 7206]]
              precision    recall  f1-score   support

        Real       0.97      0.96      0.96      7006
        Fake       0.96      0.97      0.97      7421

    accuracy                           0.97     14427
   macro avg       0.97      0.97      0.97     14427
weighted avg       0.97      0.97      0.97     14427



In [30]:
cv_svm = cross_val_score(svm, X_train, y_train, cv=5, scoring="f1", n_jobs=-1)
print(cv_svm.mean(), "±", cv_svm.std())

0.9665232997779212 ± 0.0014835257905028857


Saving the model

In [31]:
import joblib

joblib.dump(svm, "svm_model.joblib")
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")

['tfidf_vectorizer.joblib']

In [79]:
sample = ["Breaking news: President caught in massive scandal"]
sample_clean = [preprocess_text(s) for s in sample]
sample_vec = vectorizer.transform(sample_clean)

pred = svm.predict(sample_vec)
print("Fake" if pred[0] == 1 else "Real")

Fake


In [80]:
feature_names = vectorizer.get_feature_names_out()


coefs = svm.calibrated_classifiers_[0].estimator.coef_[0]

top_fake = np.argsort(coefs)[-15:][::-1]
top_real = np.argsort(coefs)[:15]

print("Top 15 words pointing to FAKE:")
for i in top_fake:
    print(f"  {feature_names[i]:25s} {coefs[i]:.4f}")

print()
print("Top 15 words pointing to REAL:")
for i in top_real:
    print(f"  {feature_names[i]:25s} {coefs[i]:.4f}")

Top 15 words pointing to FAKE:
  image                     6.6962
  october                   4.8351
  video                     4.7796
  images                    3.8172
  breaking                  3.7126
  november                  3.6414
  entire story              3.6077
  saidthe                   3.5061
  hillary                   3.4532
  breitbart news            3.3067
  obama                     3.0443
  flickr                    2.5628
  dc                        2.5227
  rt                        2.4841
  getty                     2.4709

Top 15 words pointing to REAL:
  reuters                   -15.8785
  breitbart                 -9.1833
  york times                -4.7901
  com                       -4.7596
  said                      -4.6671
  twitter                   -4.6616
  washington reuters        -4.5305
  president donald          -4.4171
  follow                    -4.1113
  pic twitter               -3.8418
  pic                       -3.1936
  ms           